In [1]:
import numpy as np
import time

import torch
from torch.autograd import Variable
import torch.utils.data as utils
from neural_net import NeuralNet
from NN_utils import read_labels, read_data

In [2]:
# Load data
#x_name = 'data/train_data_GS.csv' 
y_name = 'data/response.csv'

#X = read_data(x_name)
y = read_labels(y_name)

X = np.load('data/train_500pcs.npy')

print(X.shape)
print(y.shape)

(6000, 500)
(6000,)


In [3]:
# Split into train and validation
# such that nb of class in train and validation are equal
percent = 0.15
idx_true = np.random.permutation([i for i in range(len(y)) if y[i]==1]) # permute so that choice is random when taking the first n elements
idx_false = np.random.permutation([i for i in range(len(y)) if y[i]==0])
idx_val = np.append(idx_true[:int(len(idx_true)*percent)],idx_false[:int(len(idx_false)*percent)])
idx_train = np.append(idx_true[int(len(idx_true)*percent):],idx_false[int(len(idx_false)*percent):])

X_train = X[idx_train,:]
y_train = y[idx_train]
X_val = X[idx_val,:]
y_val = y[idx_val]

print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)

Train data shape:  (5100, 500)
Train labels shape:  (5100,)
Validation data shape:  (900, 500)
Validation labels shape:  (900,)


In [4]:
X_train, y_train = torch.from_numpy(X_train).type(torch.FloatTensor), torch.from_numpy(y_train).type(torch.LongTensor)
X_val, y_val = torch.from_numpy(X_val).type(torch.FloatTensor), torch.from_numpy(y_val).type(torch.LongTensor)

traindataset = utils.TensorDataset(X_train, y_train)
trainloader = utils.DataLoader(traindataset, batch_size=50, shuffle=True)

val_batch_size = 50
valdataset = utils.TensorDataset(X_val, y_val)
valloader = utils.DataLoader(valdataset, batch_size=val_batch_size, shuffle=True)

In [5]:
# Choice of hyperparameters
n_combinations = 20
lr = 10**(np.random.uniform(-2,-1,n_combinations)) # Random numbers between 10^-10 and 10^0 (log-scale)
reg = 10**(np.random.uniform(-10,-7,n_combinations)) # Random numbers between 10^-10 and 10^0 (log-scale)
momentum = 0.95
criterion = torch.nn.CrossEntropyLoss()
epochs = 1

In [6]:
# Try different learning rates and regularizations
for n in range(n_combinations):
    net = NeuralNet(n_input_channels=X_train.shape[1]) # Reinitialize neural net
    optimizer = torch.optim.SGD(net.parameters(), lr = lr[n], weight_decay = reg[n], momentum = momentum)
    # Training and validation
    for e in range(epochs):
        for data, labels in iter(trainloader):
            #transofrm inputs and outputs into Variable 
            inputs, targets = Variable(data), Variable(labels)

            #set gradient to zero
            optimizer.zero_grad()

            # forward pass
            out = net.forward(inputs)

            loss = criterion(out, targets)
            loss.backward()
            optimizer.step()


    # Validation accuracy
    accuracy = 0
    for ii, (data, labels) in enumerate(valloader):
        out = net.predict(Variable(data))
        _, prediction = torch.max(out, 1)
        pred_y = prediction.data.numpy().squeeze()
        target_y = (labels.numpy()).data
        accuracy += sum(pred_y == target_y)/val_batch_size

    print("val_acc: {:.4f}..".format(accuracy/(len(valloader))),
          "lr: {:.6e}..".format(lr[n]),
          "reg: {:.6e}..".format(reg[n]),
          "momentum: {:.4f}..".format(momentum)
          )

val_acc: 0.8433.. lr: 1.255184e-02.. reg: 4.921959e-09.. momentum: 0.9500..
val_acc: 0.7789.. lr: 6.119068e-02.. reg: 5.201568e-09.. momentum: 0.9500..
val_acc: 0.5000.. lr: 1.198604e-02.. reg: 2.657266e-10.. momentum: 0.9500..
val_acc: 0.5011.. lr: 2.129107e-02.. reg: 7.072954e-10.. momentum: 0.9500..
val_acc: 0.9078.. lr: 2.521031e-02.. reg: 1.814566e-10.. momentum: 0.9500..
val_acc: 0.8711.. lr: 9.943488e-02.. reg: 4.820876e-10.. momentum: 0.9500..
val_acc: 0.7556.. lr: 5.672386e-02.. reg: 1.766446e-09.. momentum: 0.9500..
val_acc: 0.8156.. lr: 2.188348e-02.. reg: 1.363405e-08.. momentum: 0.9500..
val_acc: 0.5000.. lr: 1.128453e-02.. reg: 4.085269e-09.. momentum: 0.9500..
val_acc: 0.8489.. lr: 5.905495e-02.. reg: 4.508896e-10.. momentum: 0.9500..
val_acc: 0.5056.. lr: 1.870326e-02.. reg: 2.703803e-10.. momentum: 0.9500..
val_acc: 0.8122.. lr: 1.838784e-02.. reg: 1.504048e-09.. momentum: 0.9500..
val_acc: 0.8722.. lr: 4.786675e-02.. reg: 3.308049e-10.. momentum: 0.9500..
val_acc: 0.8